In [6]:
# imports

# set the configuration
import configparser
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pathlib import Path

from models import UNet
from models.deeplab import get_deeplab


import seg_data

import torch.nn.functional as F

In [7]:
# class CustomImageDataset(Dataset):
#     def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
#         self.img_labels = pd.read_csv(annotations_file)
#         self.img_dir = img_dir
#         self.transform = transform
#         self.target_transform = target_transform

#     def __len__(self):
#         return len(self.img_labels)

#     def __getitem__(self, idx):
#         img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#         image = read_image(img_path)
#         label = self.img_labels.iloc[idx, 1]
#         if self.transform:
#             image = self.transform(image)
#         if self.target_transform:
#             label = self.target_transform(label)
#         return image, label


In [8]:
## Dice coefficient

import torch
from torch import Tensor


def dice_coeff(input: Tensor, target: Tensor, reduce_batch_first: bool = False, epsilon: float = 1e-6):
    # Average of Dice coefficient for all batches, or for a single mask
    assert input.size() == target.size()
    assert input.dim() == 3 or not reduce_batch_first

    sum_dim = (-1, -2) if input.dim() == 2 or not reduce_batch_first else (-1, -2, -3)

    inter = 2 * (input * target).sum(dim=sum_dim)
    sets_sum = input.sum(dim=sum_dim) + target.sum(dim=sum_dim)
    sets_sum = torch.where(sets_sum == 0, inter, sets_sum)

    dice = (inter + epsilon) / (sets_sum + epsilon)
    return dice.mean()


def multiclass_dice_coeff(input: Tensor, target: Tensor, reduce_batch_first: bool = False, epsilon: float = 1e-6):
    # Average of Dice coefficient for all classes
    return dice_coeff(input.flatten(0, 1), target.flatten(0, 1), reduce_batch_first, epsilon)


def dice_loss(input: Tensor, target: Tensor, multiclass: bool = False):
    # Dice loss (objective to minimize) between 0 and 1
    fn = multiclass_dice_coeff if multiclass else dice_coeff
    return 1 - fn(input, target, reduce_batch_first=True)


In [16]:
def evaluate(model,model_name, dataloader, device, amp = True):
    """Evaluate the validation set
    
    """
    model.eval()
    num_val_batches = len(dataloader)
    dice_score = 0

    # iterate over the validation set
    with torch.autocast(device.type if device.type != 'mps' else 'cpu', enabled=amp):
        for batch in dataloader:
            images, mask_true = batch[0], batch[1]

            # move images and labels to correct device and type
            images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            mask_true = mask_true.to(device=device, dtype=torch.float32)

            # predict the mask
            mask_pred = model(images)

            if model_name=="deeplab":
                mask_pred = mask_pred['out']

            if model.n_classes == 1:
                assert mask_true.min() >= 0 and mask_true.max() <= 1, 'True mask indices should be in [0, 1]'
                mask_pred = (F.sigmoid(mask_pred) > 0.5).float()
                # compute the Dice score
                dice_score += dice_coeff(mask_pred, mask_true, reduce_batch_first=False)
            else:
                assert mask_true.min() >= 0 and mask_true.max() < model.n_classes, 'True mask indices should be in [0, n_classes['
                # convert to one-hot format
                mask_true = F.one_hot(mask_true.argmax(dim=1).to(torch.long), model.n_classes).permute(0, 3, 1, 2).float()
                mask_pred = F.one_hot(mask_pred.argmax(dim=1), model.n_classes).permute(0, 3, 1, 2).float()
                # compute the Dice score, ignoring background
                dice_score += multiclass_dice_coeff(mask_pred[:, 1:], mask_true[:, 1:], reduce_batch_first=False)

    model.train()
    return dice_score / max(num_val_batches, 1)

In [10]:



## Read the config
def read_ini(file_path):
    config = configparser.ConfigParser()
    config.read(file_path)
    return config

 
config = read_ini("./train_config.ini")

for section in config.sections():
    print(section)
    for key in config[section]:
        print((key, config[section][key]))


img_path = config["DIR"]["image_dir"]
mask_path =config["DIR"]["mask_dir"]
checkpoint_path = config["DIR"]["checkpoint_path"]

model_name = config["PARAMS"]['model']
scale = int(config["PARAMS"]["scale"])

learning_rate = float(config["PARAMS"]['learning_rate'])
batch_size = int(config["PARAMS"]['batch_size'])
val_percent = int(config["PARAMS"]['val_percent'])

# number of classes
n_classes = int(config["PARAMS"].get('n_classes',False))
# Class weight, if not specified, assign None
class_weights = config["PARAMS"].get('class_weights',False)
if class_weights:
    
    class_weights = eval(class_weights)
    assert len(class_weights) == n_classes, "The length of class weights: {} should equal to the number of classes: {}".format(len(class_weights),n_classes)
    class_weights_tensor = torch.tensor(class_weights)
    class_weights = True

epochs = int(config["PARAMS"]['epochs'])



# n_classes = config

# config.get("UNET")
if model_name == "unet" and "UNET" in config:
    bilinear = config["UNET"]["bilinear"]
    

DIR
('image_dir', './data/test_10/img')
('mask_dir', './data/test_10/mask_pl/')
('checkpoint_path', './checkpoint/')
PARAMS
('model', 'deeplab')
('scale', '10')
('n_classes', '2')
('epochs', '15')
('learning_rate', '0.001')
('val_percent', '20')
('batch_size', '4')
UNET
('bilinear', 'TRUE')


In [11]:
# Read and init datasets
dataset = seg_data.segDataset(img_path = img_path, 
    mask_path = mask_path,
    scale=20)

# Split and create dataloader
l=dataset.__len__()
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-int(np.ceil(l*val_percent/100))])
dataset_val = torch.utils.data.Subset(dataset, indices[int(-np.ceil(l*val_percent/100)):])


train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                        shuffle=True, num_workers=0, pin_memory=True)


test_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size,
                                        shuffle=False, num_workers=0, pin_memory=True)

print(f'''Dataset info:
        Image folder: {img_path}
        input image scale: {scale}
        Mask folder: {mask_path}
        Dataset length: {l}
        Validation set percentage: {val_percent}
        Batch size: {batch_size}
    ''')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'''System info:
        Using device: {device}
        CPU cores: {os.cpu_count()}
        GPU count: {torch.cuda.device_count()}
'''
)


# init model and device



### Unet ###
if model_name =='unet':
    model = UNet(n_channels=3, n_classes=n_classes, bilinear=bilinear)
elif model_name =="deeplab":
    model = get_deeplab(n_classes)
    
# switch NCHW to NHWC
model = model.to(memory_format=torch.channels_last)
model.to(device=device)

# Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
optimizer = optim.RMSprop(model.parameters(),
                            lr=learning_rate, foreach=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5)  # goal: maximize Dice score
# grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)

# Loss function
if class_weights:
    print("Using class weight in loss: {}".format(class_weights_tensor))
    class_weights_tensor = class_weights_tensor.to(device=device)
    criterion = nn.CrossEntropyLoss(weight= class_weights_tensor) if n_classes > 1 else nn.BCEWithLogitsLoss()
# Loss function with class weight
else:
    print("No class weight in loss")
    criterion = nn.CrossEntropyLoss() if n_classes > 1 else nn.BCEWithLogitsLoss()



global_step = 0

scaler=None

# Learning rate warm up
warmup_factor = 1.0 / 1000
warmup_iters = min(1000, len(train_loader) - 1)
warm_up_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer, start_factor=warmup_factor, total_iters=warmup_iters
)

print(
    f'''Model info:
        Model name: {model_name}
        
        Output channels: {n_classes}
        Epochs: {epochs}
        Learning Rate: {learning_rate}
        
''')
 
#{"Bilinear" if model.bilinear else "Transposed conv"} upscaling



# if args.load:
#     state_dict = torch.load(args.load, map_location=device)
#     del state_dict['mask_values']
#     model.load_state_dict(state_dict)
#     logging.info(f'Model loaded from {args.load}')




Dataset info:
        Image folder: ./data/test_10/img
        input image scale: 10
        Mask folder: ./data/test_10/mask_pl/
        Dataset length: 10
        Validation set percentage: 20
        Batch size: 4
    
System info:
        Using device: cuda
        CPU cores: 16
        GPU count: 1



In [12]:
# init model and device



### Unet ###
if model_name =='unet':
    model = UNet(n_channels=3, n_classes=n_classes, bilinear=bilinear)
elif model_name =="deeplab":
    model = get_deeplab(n_classes)
    
# switch NCHW to NHWC
model = model.to(memory_format=torch.channels_last)
model.to(device=device)

# Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
optimizer = optim.RMSprop(model.parameters(),
                            lr=learning_rate, foreach=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5)  # goal: maximize Dice score
# grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)

# Loss function
if class_weights:
    print("Using class weight in loss: {}".format(class_weights_tensor))
    class_weights_tensor = class_weights_tensor.to(device=device)
    criterion = nn.CrossEntropyLoss(weight= class_weights_tensor) if n_classes > 1 else nn.BCEWithLogitsLoss()
# Loss function with class weight
else:
    print("No class weight in loss")
    criterion = nn.CrossEntropyLoss() if n_classes > 1 else nn.BCEWithLogitsLoss()



global_step = 0

scaler=None

# Learning rate warm up
warmup_factor = 1.0 / 1000
warmup_iters = min(1000, len(train_loader) - 1)
warm_up_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer, start_factor=warmup_factor, total_iters=warmup_iters
)

print(
    f'''Model info:
        Model name: {model_name}
        
        Output channels: {n_classes}
        Epochs: {epochs}
        Learning Rate: {learning_rate}
        
''')
 
#{"Bilinear" if model.bilinear else "Transposed conv"} upscaling



# if args.load:
#     state_dict = torch.load(args.load, map_location=device)
#     del state_dict['mask_values']
#     model.load_state_dict(state_dict)
#     logging.info(f'Model loaded from {args.load}')




c:\Users\csyic\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\csyic\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Total: 60991319, trainable: 16125954
No class weight in loss
Model info:
        Model name: deeplab
        
        Output channels: 2
        Epochs: 15
        Learning Rate: 0.001
        



In [17]:
writer = SummaryWriter()

# 5. Begin training
for epoch in range(1, epochs + 1):
    model.train()
    epoch_loss = 0
    print("Training epoch: {}/{}".format(epoch,epochs))
    for batch in train_loader:
        images, true_masks = batch[0], batch[1]

        # assert images.shape[1] == model.n_channels, \
        #     f'Network has been defined with {model.n_channels} input channels, ' \
        #     f'but loaded images have {images.shape[1]} channels. Please check that ' \
        #     'the images are loaded correctly.'
        

        images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
        true_masks = true_masks.to(device=device, dtype=torch.float32)
        

        masks_pred = model(images)

        if model_name =="deeplab":
            masks_pred = masks_pred['out']

        print(true_masks.shape, masks_pred.shape)
        if model.n_classes == 1:
            loss = criterion(masks_pred.squeeze(1), true_masks.float())
            loss += dice_loss(F.sigmoid(masks_pred.squeeze(1)), true_masks.float(), multiclass=False)
        else:
            loss = criterion(masks_pred, true_masks)
            loss += dice_loss(
                F.softmax(masks_pred, dim=1).float(),
                F.one_hot(true_masks.argmax(dim=1), model.n_classes).permute(0, 3, 1, 2).float(),
                multiclass=True
            )
        writer.add_scalar('Loss/train', loss, epoch)
        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        if epoch == 0:
            warm_up_scheduler.step()
        
        # Update the global step and epoch loss
        global_step += 1
        epoch_loss += loss.item()

    
    # Validation stage
    val_score = evaluate(model,model_name, test_loader, device)
    writer.add_scalar('Loss/Valid', loss, epoch)
    scheduler.step(val_score)    

 
    Path(checkpoint_path).mkdir(parents=True, exist_ok=True)
    state_dict = model.state_dict()
    # state_dict['mask_values'] = dataset.mask_values
    torch.save(model, str(checkpoint_path + '/{}_checkpoint_epoch{}.pth'.format(model_name, epoch)))
    print(f'Checkpoint {epoch} saved! in {checkpoint_path}')
writer.flush()
writer.close()

Training epoch: 1/15
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
Checkpoint 1 saved! in ./checkpoint/
Training epoch: 2/15
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
Checkpoint 2 saved! in ./checkpoint/
Training epoch: 3/15
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
Checkpoint 3 saved! in ./checkpoint/
Training epoch: 4/15
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
Checkpoint 4 saved! in ./checkpoint/
Training epoch: 5/15
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
Checkpoint 5 saved! in ./checkpoint/
Training epoch: 6/15
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164, 247])
torch.Size([4, 2, 164, 247]) torch.Size([4, 2, 164,

In [129]:
(F.softmax(masks_pred, dim=1).float()).shape
F.one_hot(true_masks, model.n_classes).permute(0, 3, 1, 2).float(),

RuntimeError: one_hot is only applicable to index tensor.

In [39]:
cv2.imshow("asd",((true_masks.permute(0, 2,3,1))[0,...,1].cpu().detach().numpy()))

: 

In [13]:
(((true_masks.permute(0, 2,3,1))[0,...,1].cpu().detach().numpy())==1).sum()

3961

In [90]:
F.one_hot(true_masks.argmax(dim=1), model.n_classes).permute(0, 3, 1, 2).float()
mask_pred = F.one_hot(masks_pred.argmax(dim=1), model.n_classes).permute(0, 3, 1, 2).float()